In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings(action='ignore')
PATH = 'data/'

In [2]:
train_qua = pd.read_csv(PATH+'train_quality_data.csv', thousands = ',')
train_prob = pd.read_csv(PATH+'train_problem_data.csv')

train_qua = train_qua.drop_duplicates()
train_qua = train_qua.reset_index(drop=True)

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,4.0,0,0,0,0,4,0,0
2,20201130210000,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0,0,0,8,0,0
3,20201130210000,10000,05.15.2138,0.0,0,0.0,0,0,8.0,0,0,0,0,8,0,0
4,20201104110000,10002,05.15.2138,0.0,0,0.0,0,0,0.0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284188,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0.0,0,0,0,0,17,0,0
284189,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,2.0,0,0,0,0,17,0,0
284190,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,1.0,0,0,0,0,17,0,0
284191,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,3.0,0,0,0,0,17,0,0


In [12]:
temp = []
for time in train_qua['time'].astype(str):
    tmp = time[0:8]
    temp.append(tmp)
    
train_qua['time'] = temp

In [18]:
temp = []
for time in train_prob['time'].astype(str):
    tmp = time[0:8]
    temp.append(tmp)
    
train_prob['time'] = temp

In [29]:
temp = []
train_qua['temp'] = train_qua['time'].astype(str) + train_qua['user_id'].astype(str)
train_prob['temp'] = train_prob['time'].astype(str) + train_prob['user_id'].astype(str)

for n in tqdm(train_qua['temp']):
    if n in train_prob['temp'].unique():
        temp.append(1)
    else:
        temp.append(0)
        
train_qua['target'] = temp

  0%|          | 0/284193 [00:00<?, ?it/s]

In [30]:
train_qua['target'].value_counts()

0    273681
1     10512
Name: target, dtype: int64

In [ ]:
test_qua = pd.read_csv(PATH+'test_quality_data.csv', thousands = ',')

test_qua = test_qua.drop_duplicates()
test_qua = test_qua.reset_index(drop=True)
test_qua

In [ ]:
temp = []
for time in test_qua['time'].astype(str):
    tmp = time[0:8]
    temp.append(tmp)
    
test_qua['time'] = temp

In [43]:
train_x = train_qua.iloc[:, 3:16]
train_y = train_qua['target']

In [47]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
#                 'objective'     : 'tweedie',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed'          : 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x, train_y):

    # split train, validation set
    X = train_x.loc[train_idx]
    y = train_y.loc[train_idx]
    valid_x = train_x.loc[val_idx]
    valid_y = train_y.loc[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 5
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 8410, number of negative: 218944
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004854 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1865
[LightGBM] [Info] Number of data points in the train set: 227354, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036991 -> initscore=-3.259395
[LightGBM] [Info] Start training from score -3.259395
Training until validation scores don't improve for 5 rounds
[20]	valid_0's auc: 0.747113	valid_0's pr_auc: 0.184877
[40]	valid_0's auc: 0.755974	valid_0's pr_auc: 0.210358
[60]	valid_0's auc: 0.76137	valid_0's pr_auc: 0.222319
[80]	valid_0's auc: 0.765275	valid_0's pr_auc: 0.230167
[100]	valid_0's auc: 0.767784	valid_0's pr_auc: 0.238709
[120]	valid_0's auc: 0.770695	valid_0's pr_auc: 0.247544
[140]	valid_0's auc: 0.772242	valid_0's p

In [48]:
print(np.mean(auc_scores))

0.7848355482276042
